In [1]:
import clean_database
import DataAugmentation
import Feature_Engineering as fe
import train_test_validation as ttv
import graficar
import linear
from linear import Linear_Model
import logistic
import mixture

#from mlp import MultilayerPerceptron
import numpy as np
import jax.numpy as jnp
import y_hot as hot
from mlp_mlflow import MultilayerPerceptron
import mlflow
import importlib

Limpieza de la base de datos
=====================================

In [2]:
datos_bajos,datos=clean_database.Clean_Data()

3364 7882


c:\Users\52333\Documents\doctorado\ml-md\proyecto\clean_database.py:375: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfd=d1.append(d2)


Data augmentation para la clase de datos '1' (alumnos con bajo desempeño escolar)
=====================================================================================

In [8]:
datos_aumentados=DataAugmentation.Smote(datos_bajos)
print(datos_aumentados.shape)
datos=np.concatenate((datos,datos_aumentados),axis=0)
#datos=datos[np.random.permutation(datos.shape[0]),:]
x,y=datos[:,1:],datos[:,0]
np.savetxt('datosy.csv',datos,delimiter=',')

(3364, 96)


Feature Engineering (Backwards selection)
===============================================

In [9]:
importlib.reload(fe)
x1,x2=fe.separate(x,y) #tenemos que separar los features en
size=x1.shape[1]
min_group=70 #son los features con los que nos queremos quedar
x1,x2,ganadores=fe.sequential_backwards(size,x1,x2,min_group)
x=x[:,ganadores]
x_train, x_test, x_v,y_train, y_test, y_v=ttv.split_data(x,y)

estas son las muestras en casda clase [[0.35       0.25       0.33333333 ... 0.4040404  1.         0.66666667]
 [0.15       0.16666667 0.08333333 ... 0.         0.         0.66666667]
 [0.1        0.         0.16666667 ... 0.4040404  0.375      0.83333333]
 ...
 [0.21052632 0.24677825 0.08655508 ... 0.455717   0.2693305  0.67311017]
 [0.25268668 0.23342058 0.16666667 ... 0.48484848 0.67473825 0.66666667]
 [0.18602135 0.12786714 0.16666667 ... 0.44166418 0.82540214 0.66666667]]
Los índices ganadores fueron: (4, 5, 14, 15, 16, 17, 18, 19, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 91, 92, 93)


Multilayer Perceptron
======================================


In [16]:
x_trainp,y_trainp=jnp.transpose(x_train),jnp.transpose(y_train)
layers=[x_trainp.shape[0],30,30,2]
lr=0.01
labels=jnp.array([0,1])
k_clases=2
samples=x.shape[0]
y_hotp=jnp.transpose(hot.one_hot(y_train,2))
stop=0.00001
max_steps=10000
x_valp,y_valp,y_hot_valp,samples_val=jnp.transpose(x_v),jnp.transpose(y_v),jnp.transpose(hot.one_hot(y_v,2)),x_v.shape[0]
run_name='MLP'
mlp_mlflow=MultilayerPerceptron(layers,lr,labels,k_clases,samples,x_trainp,y_trainp,y_hotp,stop,max_steps)
loss,recall_list,precision_list,loss_list,precision_list_val,recall_list_val,loss_list_val=mlp_mlflow.modelo(mlp_mlflow.weights,max_steps,x_trainp,y_trainp,y_hotp,lr,k_clases,samples,labels,stop,x_valp,y_valp,y_hot_valp,samples_val,run_name)


TRAINING: In the epoch     0 the loss is 2.11892 and the accuracy is 0.42861
VALIDATION: In the epoch     0 the loss is 2.04970 and the accuracy is 0.41881
TRAINING:In the epoch     0 the precission is 0.23214 and the recall is 0.22763
VALIDATION:In the epoch     0 the precission is 0.32345 and the recall is 0.29692
TRAINING: In the epoch    10 the loss is 1.60480 and the accuracy is 0.49982
VALIDATION: In the epoch    10 the loss is 1.56820 and the accuracy is 0.49684
TRAINING: In the epoch    20 the loss is 1.31062 and the accuracy is 0.53490
VALIDATION: In the epoch    20 the loss is 1.29404 and the accuracy is 0.52847
TRAINING: In the epoch    30 the loss is 1.11625 and the accuracy is 0.55325
VALIDATION: In the epoch    30 the loss is 1.10935 and the accuracy is 0.55377
TRAINING: In the epoch    40 the loss is 0.97168 and the accuracy is 0.56598
VALIDATION: In the epoch    40 the loss is 0.96997 and the accuracy is 0.56263
TRAINING: In the epoch    50 the loss is 0.85598 and the a

Linear Model
=================

In [48]:
dim=int(x_train.shape[1])
y_trainl=y_train
y_vl=y_v

for i in range(9740):
    if y_train[i]==0:
        y_trainl[i]==-1
for i in range(1624):
    if y_v[i]==0:
        y_vl[i]=-1

x_trainl=jnp.array(x_train)
y_trainl=jnp.array(y_trainl)
x_vl=jnp.array(x_v)
y_vl=jnp.array(y_vl)
labels=jnp.array([0,1])
k_classes=2
n_steps=1000
lr=1e-6
samples=x_trainl.shape[0]
#linear model
#importlib.reload(linear)
modelo=Linear_Model(dim)
theta = modelo.generate_theta() #inicializamos theta con valores aleatorios
print(x_trainl.shape, y_trainl.shape)
y_trainl=jnp.reshape(y_trainl,(9740,1))
print(x_trainl.shape, y_trainl.shape,dim)
#theta = modelo.gradient_descent(theta, x_trainl, y_trainl, 10, lr = 0.01)
#y_hat=modelo.estimate_grsl(x_trainl,theta)





(9740, 79) (9740,)
(9740, 79) (9740, 1) 79


In [ ]:
run_name='lineal'
y_hatl,y_hat_vall=modelo.model(theta, x_trainl, y_trainl, lr,n_steps,x_vl,y_vl,run_name)

este es lr
este es el error en el paso i 0 43401.574
este es el error en el paso i 1 35594.105
este es el error en el paso i 2 32194.805
este es el error en el paso i 3 30336.92
este es el error en el paso i 4 29054.17
este es el error en el paso i 5 28017.623
este es el error en el paso i 6 27112.299
este es el error en el paso i 7 26295.375
este es el error en el paso i 8 25548.832
este es el error en el paso i 9 24862.883
este es el error en el paso i 10 24230.96
este es el error en el paso i 11 23648.262
este es el error en el paso i 12 23110.244
este es el error en el paso i 13 22613.117
este es el error en el paso i 14 22153.184
este es el error en el paso i 15 21727.514
este es el error en el paso i 16 21333.047
este es el error en el paso i 17 20967.111
este es el error en el paso i 18 20627.445
este es el error en el paso i 19 20311.713
este es el error en el paso i 20 20017.963
este es el error en el paso i 21 19744.246
este es el error en el paso i 22 19489.066
este es el er

KeyboardInterrupt: 

In [ ]:
#Ridge
l=0
importlib.reload(linear)

incremento=0.1
run_name='Ridge'
samples=500
x_train_r=x_train[:500]
y_train_r=y_train[:500]
x_vr=x_v[:500]
y_vr=y_v[:500]
y_hatR,y_hatR_v,precision_list=modelo.model_ridge(x_train_r,y_train_r,l,incremento,samples,k_classes,labels,x_vr,y_vr,run_name)



holaa


Logistic
================

0.1
lambda 0.1
-7713.833984375
-5637.595703125
-4684.83740234375
-4115.771484375
0.8593429327011108


Mixture of Gaussians
===========================

In [57]:
importlib.reload(mixture)

clases=int(2)


x11,x12=fe.separate(x_train,y_train)
print(x11.shape,x12.shape)
samples1=int(x11.shape[0])
samples2=int(x12.shape[0])
features=x12.shape[1]
print(features)

#valores para inicializar el paso EM
p=np.array([0.3,0.7])
mu1=0.4*np.ones(features)
mu2=0.3*np.ones(features)
sigma1=np.identity(features)
sigma2=np.identity(features)
#sigma1=([0.1,0.2],[0.3,0.4])
#sigma2=([0.4,0.5],[0.6,0.8])
mu1,sigma1,mu2,sigma2,p,Nk1,Nk2=mixture.solve(clases,features,samples1,samples2,p,mu1,mu2,sigma1,sigma2,x11,x12)
samples=x_train.shape[0]
print('estps son los mu1 y mu2',mu1,mu2)
y_hat,precision=mixture.get_prediction(x_train,mu1,sigma1,mu2,sigma2,features,samples,y_train)
precision


(4450, 79) (5290, 79)
79
1
esta es la gausiana [4.40639283e-35 3.69338316e-35 8.83044083e-35 9.61194478e-35
 3.51080668e-35 4.74796790e-35 4.91426392e-35 1.61788793e-35
 7.71680364e-35 5.13401204e-35] [7.04349864e-36 3.58017167e-35 1.88419400e-35 5.21552437e-36
 1.42494976e-35 1.98734569e-35 9.97230807e-36 3.18129622e-35
 1.07165102e-35 3.82274357e-36]
este es el denominador [2.15337267e-35 1.41662407e-34 2.55097070e-34 5.03642953e-35
 1.45192197e-34 1.98893127e-34 2.15924746e-34 8.89227726e-36
 2.24155787e-34 2.38506630e-34]
estos son los nk 992.1646012924134 2248.314808503209
[0.25855738 0.21371023 0.20622078 0.88150117 0.75262457 0.61537186
 0.89845326 0.51959195 0.62186534 0.15022022 0.15331979 0.18096505
 0.54473886 0.7037893  0.62974335 0.38899897 0.58202198 0.38594292
 0.5821853  0.00124194 0.00107264 0.00831973 0.11107336 0.01123976
 0.0217192  0.13837279 0.41056875 0.46528621 0.6525795  0.68748489
 0.61394998 0.71338793 0.76970508 0.76954701 0.62426307 0.38706444
 0.74508844 0

0.19187289476394653

In [56]:
clases=int(2)

importlib.reload(mixture)
x11=X[:27940]
x12=X[27940:]
print(x11.shape,x12.shape)
samples1=int(x11.shape[0])
samples2=int(x12.shape[0])
features=x12.shape[1]
print(x11)
print('este es samples1',samples1)

#valores para inicializar el paso EM
p=np.array([0.3,0.7])
mu1=0.3*np.ones(features)
mu2=0.4*np.ones(features)
sigma1=np.identity(features)
sigma2=np.identity(features)
#sigma1=([0.1,0.2],[0.3,0.4])
#sigma2=([0.4,0.5],[0.6,0.8])
mu1,sigma1,mu2,sigma2,p,Nk1,Nk2=mixture.solve(clases,features,samples1,samples2,p,mu1,mu2,sigma1,sigma2,x11,x12)
samples=x_v.shape[0]
y_hat,precision=mixture.get_prediction(x_v,mu1,sigma1,mu2,sigma2,features,samples,y_v)
precision

(27940, 48) (27940, 48)
[[-2.18886782 -0.31223998 -0.88368219 ...  0.61870714  0.63077574
   0.61191785]
 [-2.18886782 -0.30716892 -0.84401342 ...  0.01973181  0.020036
   0.01987001]
 [-2.18886782 -0.30716892 -0.84401342 ... -0.01239973 -0.01220737
  -0.01238624]
 ...
 [ 0.09743488 -0.26491071 -0.39904913 ... -0.01050167 -0.01080887
  -0.01080676]
 [ 0.75084546 -0.23797973 -0.85527783 ... -0.01293859 -0.01262878
  -0.01334894]
 [-0.82954904 -0.30716892 -0.81618606 ... -0.01124189 -0.01095955
  -0.01135704]]
este es samples1 27940
1
esta es la gausiana [8.59820929e-24 2.23353285e-23 3.83186335e-24 6.84461873e-24
 4.57453769e-24 6.92356506e-24 1.18715519e-24 1.55721063e-23
 1.35123745e-27 1.02195373e-24] [1.75338098e-22 3.14519706e-28 2.58867977e-23 4.02574806e-26
 2.01061327e-22 4.07960167e-23 1.54892641e-25 1.01526005e-23
 2.54297188e-26 1.63650653e-23]
este es el denominador [8.74476303e-24 8.56029234e-24 1.45352934e-24 2.53033364e-24
 1.66061943e-24 2.56627506e-24 4.89874150e-25 6.1

c:\Users\52333\Documents\doctorado\ml-md\proyecto\mixture.py:61: RuntimeWarning: invalid value encountered in double_scalars
  gama[i]=p*gausiana[i]/denominador[i]


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [nan nan]
2


c:\Users\52333\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\linalg\linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


esta es la gausiana [nan nan nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan nan nan]
este es el denominador [nan nan nan nan nan nan nan nan nan nan]
estos son los nk nan nan
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan] [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]] [nan nan]
3
esta es la gausian

KeyboardInterrupt: 